## Assignment: Segmenting and Clustering Neighborhoods in Toronto - Part 2
#### Problem: Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. 

##### An attempt to use the geocoder library but it was too slow and no results displayed

In [13]:
# the result of the first part of the assignement; used to have the initial dataframe
import requests
from bs4 import BeautifulSoup
import pandas as pd
# Create a dataframe from the wikipedia table
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, "html.parser")
table = soup.find("table", class_ = "wikitable sortable")
postcodes=[]
boroughs=[]
neighborhoods=[]
for row in table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells)==3: #Only extract table body not heading
        postcodes.append(cells[0].find(text=True))
        boroughs.append(cells[1].find(text=True))
        neighborhoods.append(cells[2].find(text=True))
# Use lists to create the dataframe
df=pd.DataFrame(postcodes,columns=["PostalCode"])
df["Borough"]=boroughs
df["Neighborhood"]=neighborhoods
df = df[df["Borough"]!="Not assigned"]
# create a grouped dataframe by Postalcode
df = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df = df.reset_index()
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
# Clean the Neighborhood text by removing new lines "\n"
df = df.replace('\n','', regex=True)
print(df.head())

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto                      Harbourfront
3        M6A        North York  Lawrence Heights, Lawrence Manor
4        M7A  Downtown Toronto                      Queen's Park


In [14]:
# An attempt to use the grocoder, but there were no results using this library
'''import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format("M5G"))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
print(latitude, longitude)'''

'import geocoder # import geocoder\n\n# initialize your variable to None\nlat_lng_coords = None\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google(\'{}, Toronto, Ontario\'.format("M5G"))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]\nprint(latitude, longitude)'

In [15]:
import io
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
df_new=pd.read_csv(io.StringIO(s.decode('utf-8')))
df_new.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
print(df_new.head())

  PostalCode   Latitude  Longitude
0        M1B  43.806686 -79.194353
1        M1C  43.784535 -79.160497
2        M1E  43.763573 -79.188711
3        M1G  43.770992 -79.216917
4        M1H  43.773136 -79.239476


In [16]:
# Create a dataframe containig both information from the first dataframe df (Postalcode, Borough and Neighborhood)
# and from the second one df_new (Latitude and Longitude)
df_merged = df.merge(df_new, on="PostalCode", how = 'inner')
print(df_merged.head())

  PostalCode           Borough                      Neighborhood   Latitude  \
0        M3A        North York                         Parkwoods  43.753259   
1        M4A        North York                  Victoria Village  43.725882   
2        M5A  Downtown Toronto                      Harbourfront  43.654260   
3        M6A        North York  Lawrence Heights, Lawrence Manor  43.718518   
4        M7A  Downtown Toronto                      Queen's Park  43.662301   

   Longitude  
0 -79.329656  
1 -79.315572  
2 -79.360636  
3 -79.464763  
4 -79.389494  
